In [ ]:


import requests
import json
import pandas as pd

#Create an empty list for reponses
response_list = []

# Create the first connection to get started
endpoint = "https://sample.thoughtindustries.com/incoming/v2/events/courseCompletion"
headers = {"Authorization": "Bearer #Enter your API Key from thought industries here"}
response = requests.get(endpoint, headers=headers).json()
response_dict = dict(response)
# Determine if hasMore is equal to True
has_more = response_dict.get('pageInfo', {}).get('hasMore')
# Use the .get method on the dictionary to pull out the cursor value
cursor_value = str(response_dict.get('pageInfo', {}).get('cursor'))
response_list.append(cursor_value)

#loop through the rest of the pages to get the cursor values. Use the range function to determine how many pages of data you would like
for x in range(1, 20):
    next_endpoint = "https://sample.thoughtindustries.com/incoming/v2/events/courseCompletion"
    next_response_url = next_endpoint +"?cursor="+ cursor_value
    next_response = requests.get(next_response_url, headers=headers).json()
    next_response_dict = dict(next_response)
    has_more = next_response_dict.get('pageInfo', {}).get('hasMore')
    cursor_value = next_response_dict.get('pageInfo', {}).get('cursor')
    response_list.append(cursor_value)
    
    
#Set loops = 0. #First loop creates the dataframe, the subsequent ones add to it.     
loops = 0

# use the cursor values in the response list now to get the data. Convert to a JSON then to a dictionary and then to a dataframe.
for cursor in response_list:
    loops +=1
    endpoint = "https://sample.thoughtindustries.com/incoming/v2/events/courseCompletion"+"?cursor="+cursor
    headers = {"Authorization": "#Enter API Key Here"}
    response = requests.get(endpoint, headers=headers).json()
    response_dict = dict(response)
    event_dict = { key:value for (key,value) in response_dict.items() if key == "events"}
    events_unnested = event_dict.get('events', {}) 
    if loops == 1:
        course_data_df = pd.DataFrame.from_dict(events_unnested)
    else:
        to_append = pd.DataFrame.from_dict(events_unnested)
        course_data_df = course_data_df.append(to_append)

print(course_data_df)